In [1]:
# Import tables
import geopandas as gpd
new = gpd.read_file('/Users/sofia/Documents/HE_Data/WDPA_Jan2022_Public_csv/WDPA_Jan2022_Public_csv.csv')
old= gpd.read_file('/Users/sofia/Documents/HE_Data/WDPA_terrestrial+marine2021_simplified')

In [2]:
len(old)

222519

In [3]:
len(new)

269457

In [4]:
old[old['WDPA_PID']=='145114']

,WDPAID,WDPA_PID,NAME,ORIG_NA,DESIG,DESIG_E,DESIG_T,IUCN_CA,MARINE,GOV_TYP,ISO3,OWN_TYP,AREA_KM,MOL_ID,geometry
90566,145114.0,145114,Zygmanti?ki? Geomorfologinis Draustinis,Zygmanti?ki? Geomorfologinis Draustinis,Gamtinis Draustinis,Nature Reserve,National,IV,0,Federal or national ministry or agency,LTU,Not Reported,22.595064,90735,"POLYGON ((25.09160 54.40030, 25.10820 54.40220..."


In [5]:
new[new['WDPA_PID']=='145114']

,TYPE,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,...,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO3,ISO3,SUPP_INFO,CONS_OBJ,geometry
173357,Polygon,145114,145114,1,Zygmantiškių Geomorfologinis Draustinis,Zygmantiškių Geomorfologinis Draustinis,Gamtinis draustinis,Nature Reserve,National,IV,...,State Protected Areas Service under The Minist...,Not Reported,State Verified,2013,Not Reported,LTU,LTU,Not Applicable,Not Applicable,None


In [6]:
old2 = old.merge(new[['WDPA_PID','NAME', 'ORIG_NAME']],how='left',on='WDPA_PID')
len(old2)

222519

In [7]:
old2['NAME_x'] = old2['NAME_y']
old2['ORIG_NA'] = old2['ORIG_NAME']
old2 = old2.rename(columns={'NAME_x': 'NAME'})
old2 = old2.drop(columns='NAME_y')
old2 = old2.drop(columns='ORIG_NAME')
old2[old2['WDPA_PID']=='145114']

,WDPAID,WDPA_PID,NAME,ORIG_NA,DESIG,DESIG_E,DESIG_T,IUCN_CA,MARINE,GOV_TYP,ISO3,OWN_TYP,AREA_KM,MOL_ID,geometry
90566,145114.0,145114,Zygmantiškių Geomorfologinis Draustinis,Zygmantiškių Geomorfologinis Draustinis,Gamtinis Draustinis,Nature Reserve,National,IV,0,Federal or national ministry or agency,LTU,Not Reported,22.595064,90735,"POLYGON ((25.09160 54.40030, 25.10820 54.40220..."


In [10]:
# Save file in local
old2.to_file('/Users/sofia/Documents/HE_Data/WDPA_terrestrial+marine2021_simplified_v2', driver= 'ESRI Shapefile', encoding='utf-8')

In [11]:
wdpa = gpd.read_file('/Users/sofia/Documents/HE_Data/WDPA_terrestrial+marine2021_simplified_v2')

In [12]:
wdpa[wdpa['WDPA_PID']=='145114']

,WDPAID,WDPA_PID,NAME,ORIG_NA,DESIG,DESIG_E,DESIG_T,IUCN_CA,MARINE,GOV_TYP,ISO3,OWN_TYP,AREA_KM,MOL_ID,geometry
90566,145114.0,145114,Zygmantiškių Geomorfologinis Draustinis,Zygmantiškių Geomorfologinis Draustinis,Gamtinis Draustinis,Nature Reserve,National,IV,0,Federal or national ministry or agency,LTU,Not Reported,22.595064,90735,"POLYGON ((25.09160 54.40030, 25.10820 54.40220..."


### Publish layers with API

In [13]:
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

In [14]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [15]:
# Function to publish files as feature services or hosted tables

def publishCSVasFS(csv_file,csvName, gis, aol_folder_name, sharing, table): # sharing = 'None', 'groups', 'everyone' # table = 'yes', 'no'
    try:
        
        if gis.content.is_service_name_available(csvName, "featureService"):
            print(f"Service name {csvName} is available")
            #csv_file = f'./{csvName}.csv'
            csv_item = gis.content.add({}, csv_file)
            #csv_lyr = csv_item.publish(None,  {'CountryCode' : 'countryname'})
            if table == 'yes':
                csv_lyr = csv_item.publish(publish_parameters={'locationType':'none', 'type':'csv'})
            else:
                print('no table')
                csv_lyr = csv_item.publish()
                flayer_collection = FeatureLayerCollection.fromitem(csv_lyr)
                searched_flayer = flayer_collection.layers[0] 
                nber_features = searched_flayer.query(return_count_only=True)
                print(f"The service {csvName} has been published. The service has {nber_features} entries")
            print(f"Moving service {csvName} to {aol_folder_name} in ArcGIS Online...")
            # move both the csv_item and csv_lyr items into this new folder
            csv_item.move(aol_folder_name)
            csv_lyr.move(aol_folder_name)
            print(f"Service {csvName} has been moved to {aol_folder_name} in ArcGIS Online")
#             sharing, default is owner
#             csv_lyr.shared_with
            if sharing == 'None':
                csv_lyr.share(everyone=False, org=False, groups=None, allow_members_to_edit=False)
            if sharing == 'groups':
            # the group id is needed to share with agroup
                csv_lyr.share(everyone=False, org=False, groups=[group_id], allow_members_to_edit=False)
            if sharing == 'everyone':
                csv_lyr.share(everyone=True, org=False, groups=None, allow_members_to_edit=False)
            sharing_prop = csv_lyr.shared_with
            if sharing_prop['everyone']==True:
                print(f"shared with everyone")   
            else:
                print(f"not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted")            
            #not allowing deleting
            csv_lyr.protect()
            print(f"{csvName}'s protection against deletion : {csv_lyr.protected}") 
            # it is possible to check the status with csv_item.content_status
            return csv_lyr.id
        else:
            print("The service name is not available, try overwritting, appending the data or a different service name")
    except:
        print("The csv hasn't been published")

In [17]:
# Run function with the file we want to add
csv_file = '/Users/sofia/Documents/HE_Data/WDPA_terrestrial+marine2021_simplified_v2.zip'
csvName = 'WDPA_terrestrialmarine_2021_simplified_v2'
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
aol_folder_name = 'wdpa data'
group_id='46783c33c294485abe83f5e899c2f21f'
publishCSVasFS(csv_file,csvName,gis,aol_folder_name,sharing='groups', table='no')

Service name WDPA_terrestrialmarine_2021_simplified_v2 is available
no table
The service WDPA_terrestrialmarine_2021_simplified_v2 has been published. The service has 222519 entries
Moving service WDPA_terrestrialmarine_2021_simplified_v2 to wdpa data in ArcGIS Online...
Service WDPA_terrestrialmarine_2021_simplified_v2 has been moved to wdpa data in ArcGIS Online
not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted
WDPA_terrestrialmarine_2021_simplified_v2's protection against deletion : True


'a032522cc9024af99c15429d56a23de6'